In [27]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

In [182]:
def replace_tokens(text, tokenised_text, keywords, punctuation, mode):
    keyword_replacements = []
    for i, t in enumerate(keywords):
        target_word = str(t)
        # ignore if it's just punctuation
        if target_word in punctuation_list:
            continue
        # ignore if it's not the list of BERT tokens
        if target_word.lower() not in tokenised_text:
            continue
            
        target_token = target_word.lower()
        masked_index = tokenised_text.index(target_token)
        tokenised_text[masked_index] = '[MASK]'

        indexed_tokens = tokeniser.convert_tokens_to_ids(tokenised_text)
        segments_ids = [1] * len(tokenised_text)
        segments_ids[0] = 0
        segments_ids[1] = 0

        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

        # Predict all tokens
        predictions = model(tokens_tensor, segments_tensors)

        chosen_token = "X"
        while True:
            predicted_index = torch.argmax(predictions[0, masked_index]).item()
            predicted_token = tokeniser.convert_ids_to_tokens([predicted_index])[0]
            if predicted_token not in punctuation_list and predicted_token.find(target_token) < 0 and target_token.find(predicted_token) < 0:
                try:
                    ri = keyword_replacements.index(predicted_token)
                except ValueError:
                    ri = -1
                if predicted_token == 'bell':
                    print("pt: " + predicted_token)
                    print(f'ri: {ri}')
                    print(f'kr: {keyword_replacements[ri]}')
                    print(f'k: {keywords[ri]}')
                    print(f'k: {keywords[i]}')
                if ri < 0 or (ri >= 0 and keywords[i] == keywords[ri]):
                    # and keyword_replacements[ri].find(predicted_token) < 0 and predicted_token.find(keyword_replacements[ri]) < 0)
                    chosen_word = predicted_token
                    print(f'{target_word} -> {chosen_word}')
                    break
                else:
                    predictions[0,masked_index,predicted_index] = -9999
            else:
                predictions[0,masked_index,predicted_index] = -9999

        if mode == 'all':
            current_text_tokens = spacy_tokeniser(text)
            text = ' '.join([str(spacy_tokeniser(chosen_word)) if str(token) == target_word else str(token) for token in current_text_tokens])
        else:
            text = text.replace(target_word, chosen_word, 1)
            
        tokenised_text = tokeniser.tokenize(text)
        keyword_replacements.append(chosen_word)
        
    return text
    

In [90]:
model_path = "bert-base-uncased"
model = BertForMaskedLM.from_pretrained(model_path)
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
   

In [183]:
tokeniser = BertTokenizer.from_pretrained(model_path)
f = open("data/tyger_tyger.txt", "r")
text = "dummy. " + f.read()
tokenised_text = tokeniser.tokenize(text)

f = open("data/tyger_tyger_keywords.txt", "r")
keywords = f.read().split()

In [184]:
# nlp = English()
# spacy_tokeniser = nlp.Defaults.create_tokenizer(nlp)
# keywords = spacy_tokeniser(text.replace('dummy. ','',1))

In [185]:
punctuation = [
    ',',
    '.',
    '!',
    '?',
    ':',
    ';',
    '...',
    '\n', 
    '\n\n'
]

In [186]:
text = replace_tokens(text, tokenised_text, keywords, punctuation, "selection")

Tiger -> a
Tiger -> house
burning -> very
bright -> large
forests -> woods
night -> city
immortal -> one
hand -> person
eye -> country
frame -> capture
fearful -> beautiful
symmetry -> home
deep -> mist
skies -> hills
Burnt -> in
fire -> light
eyes -> love
wings -> day
dare -> shall
hand -> lord
dare -> to
seize -> be
fire -> father
shoulder -> art
art -> science
twist -> make
heart -> bones
heart -> drum
beat -> call
dread -> the
hand -> bells
dread -> the
pt: bell
ri: -1
kr: the
k: symmetry
k: feet
feet -> bell
pt: bell
ri: 32
kr: bell
k: feet
k: hammer
hammer -> ring
pt: bell
ri: 32
kr: bell
k: feet
k: chain
chain -> song
furnace -> time
brain -> cradle
dread -> the
grasp -> world
Dare -> and
deadly -> many
terror -> joy
clasp -> sing
stars -> kings
threw -> lay
down -> with
spears -> hearts
water -> praise
heaven -> thee
tears -> songs
smile -> give
work -> heart
see -> me
made -> is
Lamb -> best
thee -> them
Tiger -> a
Tiger -> house
burning -> so
bright -> small
forests -> dark
n

In [187]:
print(text.replace('dummy. ','', 1))

a house, very large, 
In the woods of the city; 
What one person or country, 
Could capture thy beautiful home?

In what distant mists or hills. 
in the light of thine love?
On what day shall he aspire?
What the lord, to be the father?

And what science, & what art,
Could make the sinews of thy bones?
And when thy drum began to call,
What the bells? & what the bell?

What the ring? what the song, 
In what time was thy cradle?
What the anvil? what the world, 
and its many joys sing! 

When the kings lay with their hearts 
And praise'd them with their songs: 
Did he give his heart to me?
Did he who is the best make thee?

a house so small, 
In the dark of the city: 
What one candle or eye,
can capture thy beautiful home?

